In [1]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import torch

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.structures import BoxMode

D:\Ott_diploma_proj_prog_files\conda\envs\ott_diploma\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    i = 0
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        i+=1
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["image_id"] = i
        record["height"] = 1080
        record["width"] = 1920

      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                #"id": i,
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [3]:

classes = ['icy_tail', 'icy_asphalt', 'powdery_snow', 'snow_drift', 'puffy_road_snow', 'snowy_road']

data_path = 'DATA/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes, thing_colors = [
        (255,255,0),(0,0,255),(0,255,0),(255,0,255), (180,165,0), (187,132,156)]) #Done by me
  
    

microcontroller_metadata = MetadataCatalog.get("category_train")

In [4]:
cfg_instance_seg = get_cfg()
cfg_instance_seg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_instance_seg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
cfg_instance_seg.MODEL.WEIGHTS = os.path.join("output", "model_final.pth")
cfg_instance_seg.DATASETS.TEST = ("category_test")
cfg_instance_seg.MODEL.ROI_HEADS.NUM_CLASSES = 6
predictor = DefaultPredictor(cfg_instance_seg)
evaluator = COCOEvaluator("category_test", cfg_instance_seg, False, output_dir="./output/eval/")

[03/11 17:28:40 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output\model_final.pth ...
[03/11 17:28:41 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [03/11 17:28:41 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[03/11 17:28:41 d2.evaluation.coco_evaluation]: Trying to convert 'category_test' to COCO format ...
[03/11 17:28:41 d2.data.datasets.coco]: Converting annotations of dataset 'category_test' to COCO format ...)
[03/11 17:28:41 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[03/11 17:28:41 d2.data.datasets.coco]: Conversion finished, #images: 36, #annotations: 239
[03/11 17:28:41 d2.data.datasets.coco]: Caching COCO format annotations at './output/eval/category_test_coco_format.json' ...


In [5]:
val_loader = build_detection_test_loader(cfg_instance_seg, "category_test")
inference_on_dataset(predictor.model, val_loader, evaluator)
#evaluator.evaluate()

[03/11 17:29:06 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |   category    | #instances   |   category   | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:------------:|:-------------|
|  icy_tail  | 21           |  icy_asphalt  | 27           | powdery_snow | 6            |
| snow_drift | 77           | puffy_road_.. | 49           |  snowy_road  | 59           |
|            |              |               |              |              |              |
|   total    | 239          |               |              |              |              |
[03/11 17:29:06 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/11 17:29:06 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[03/11 17:29:06 d2.data.common]: Serializing 36 elements to byte tensors an

C:\Users\gvo2\AppData\Roaming\Python\Python37\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/11 17:29:32 d2.evaluation.evaluator]: Inference done 1/36. Dataloading: 2.9400 s/iter. Inference: 22.5458 s/iter. Eval: 0.1991 s/iter. Total: 25.6856 s/iter. ETA=0:14:58
[03/11 17:29:37 d2.evaluation.evaluator]: Inference done 33/36. Dataloading: 0.0006 s/iter. Inference: 0.1028 s/iter. Eval: 0.0415 s/iter. Total: 0.1450 s/iter. ETA=0:00:00
[03/11 17:29:37 d2.evaluation.evaluator]: Total inference time: 0:00:04.924082 (0.158841 s / iter per device, on 1 devices)
[03/11 17:29:37 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:03 (0.103679 s / iter per device, on 1 devices)
[03/11 17:29:37 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[03/11 17:29:37 d2.evaluation.coco_evaluation]: Saving results to ./output/eval/coco_instances_results.json
[03/11 17:29:37 d2.evaluation.coco_evaluation]: Evaluating predictions with official COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluatio

OrderedDict([('bbox',
              {'AP': 68.24223892270103,
               'AP50': 79.1983922153961,
               'AP75': 71.51186016810081,
               'APs': 99.99999999999997,
               'APm': 74.04840484048404,
               'APl': 67.89845847494266,
               'AP-icy_tail': 72.16831683168317,
               'AP-icy_asphalt': 75.37534676544577,
               'AP-powdery_snow': 83.16831683168317,
               'AP-snow_drift': 63.98539094190991,
               'AP-puffy_road_snow': 44.93654300507761,
               'AP-snowy_road': 69.81951916040666}),
             ('segm',
              {'AP': 62.55946577841607,
               'AP50': 78.79077921464875,
               'AP75': 72.27218092003848,
               'APs': 80.0,
               'APm': 69.55952738130955,
               'APl': 62.042043082971276,
               'AP-icy_tail': 60.01231730315889,
               'AP-icy_asphalt': 76.73845230676915,
               'AP-powdery_snow': 74.27722772277228,
       